In [1]:
import json
#import pydevd
#pydevd.settrace('localhost', port=49309, stdoutToServer=True, stderrToServer=True)
import tensorflow as tf

import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import OrderedDict, defaultdict
from six.moves import xrange
#from __pynauty__ import graph
#import pynauty.graph
import pynauty
import time
import tensorflow as tf
import networkx as nx
import numpy as np
import pynauty as nauty
from multiprocessing import Pool
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from PatchyTools import Dataset
from PatchyTools import utils
from PatchyTools import Patchy_san

### Functions

In [2]:
mutag = Dataset.Dropbox('MUTAG')
df_edge_label = mutag.get_edge_label()
df_graph_ind = mutag.get_graph_ind()
df_adj = mutag.get_adj()
df_node_label = mutag.get_node_label()
df_node_label = pd.concat([df_node_label, df_graph_ind.graph_ind], axis=1)
del df_graph_ind

num_features = len(df_node_label.label.unique())

In [3]:
data_in_patchysan = Patchy_san.main(WIDTH_W=18, RECEPTIVE_FIELD_SIZE_K=10, datasetname='MUTAG')

In [4]:
data_in_patchysan.shape[3]

7

In [5]:
final_labels = mutag.get_graph_label().graph_label.values

# Logistic Regression

In [6]:

from sklearn.linear_model import SGDClassifier
ret =[]

for seed in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(data_in_patchysan, final_labels, 
                                                        test_size=0.10, random_state=seed)
    lr_model = SGDClassifier(loss="log", penalty="l2", max_iter=1000,tol=1e-3 )
    X_train = X_train.reshape( (X_train.shape[0],X_train.shape[1]*X_train.shape[2]*X_train.shape[3]))
    lr_model.fit(X_train,y_train)
    X_test = X_test.reshape( (X_test.shape[0],X_test.shape[1]*X_test.shape[2]*X_test.shape[3]))
    y_pred = lr_model.predict(X_test)
    ret.append(accuracy_score(y_test, y_pred))
    
    
print (np.mean(ret), np.std(ret))

0.817263157895 0.0960611531908


In [7]:
final_labels = pd.get_dummies(final_labels).values

### CNN  

In [8]:

##########################################
# Build a Multilayer Convolutional Network
###########################################

from __future__ import absolute_import, unicode_literals
import tensorflow as tf

FeatureNum = data_in_patchysan.shape[3]
PixcelSize = data_in_patchysan.shape[1]* data_in_patchysan.shape[2]

sess = tf.InteractiveSession()

W = tf.Variable(tf.zeros([PixcelSize, FeatureNum]))
b = tf.Variable(tf.zeros([FeatureNum]))

"""
Weight Initialization

To create this model, we're going to need to create a lot of weights and biases.
One should generally initialize weights with a small amount of noise for symmetry breaking,
and to prevent 0 gradients. Since we're using ReLU neurons, it is also good practice to initialize
them with a slightly positive initial bias to avoid "dead neurons." Instead of doing this repeatedly
while we build the model, let's create two handy functions to do it for us.
"""


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

"""
Convolution and Pooling

TensorFlow also gives us a lot of flexibility in convolution and pooling operations.
How do we handle the boundaries? What is our stride size? In this example,
we're always going to choose the vanilla version. Our convolutions uses a stride of one
and are zero padded so that the output is the same size as the input. Our pooling is plain old
max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.
"""


def conv2d(x, W):
    print('x.shape, W.shape', x.shape, W.shape)
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 1, 1, 1],
                          strides=[1, 1, 1, 1], padding='SAME')
"""
1st layer
patch size : 10x10 
input channel : F
output channel : 16

"""

W_conv1 = weight_variable([data_in_patchysan.shape[1],
                           data_in_patchysan.shape[2], 
                           FeatureNum, 
                           16])

b_conv1 = bias_variable([16])

x_image = tf.placeholder("float", shape=[None, 18, 10, 7]) #tf.reshape(data_in_patchysan, [-1, 18, 5, 7])
y_ = tf.placeholder("float", shape=[None,2])
temp = conv2d(x_image, W_conv1) + b_conv1
h_conv1 = tf.nn.relu(temp)

"""
1st layer
patch size : 5x5
input channel : 8
output channel : 8
"""

W_conv2 = weight_variable([18, 10, 16, 8])
b_conv2 = bias_variable([8])


h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
#h_pool2 = max_pool_2x2(h_conv2)

"""

Densely Connected Layer

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow
processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors,
multiply by a weight matrix, add a bias, and apply a ReLU.
"""

#num = np.product([i for i in h_conv2.shape[1:]])
num = 1440
h_conv2_flat = tf.reshape(h_conv2, [-1, num])

W_fc1 = weight_variable([num, 128])
b_fc1 = bias_variable([128])
h_fc1 = tf.nn.relu(tf.matmul(h_conv2_flat, W_fc1) + b_fc1)

"""
Dropout
To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder
for the probability that a neuron's output is kept during dropout. This allows us to turn dropout
on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically
handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.
"""
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

"""
Readout Layer
Finally, we add a softmax layer, just like for the one layer softmax regression above.
"""
num_final_class = data_in_patchysan.shape[1]
W_fc2 = weight_variable([128,num_final_class])
b_fc2 = bias_variable([num_final_class])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

x.shape, W.shape (?, 18, 10, 7) (18, 10, 7, 16)
x.shape, W.shape (?, 18, 10, 16) (18, 10, 16, 8)


In [15]:


"""
Training and evaluation 
TensorFlow
"""


cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#accuracy = tf.metrics.accuracy(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
sess.run(tf.initialize_all_variables())
tf.global_variables_initializer()

for i in range(2000):
    batch = data_in_patchysan
    if i % 100 == 2 and i>0:
        train_accuracy = accuracy.eval(feed_dict={
            x_image: data_in_patchysan[:5], 
            y_: final_labels[:5],
            keep_prob: .99
        })
        print ("step %d, training accuracy %g" % (i, train_accuracy))
    train_step.run(feed_dict={x_image: data_in_patchysan, y_: final_labels, keep_prob: 0.5})

# Result
print( "test accuracy %g" % accuracy.eval(feed_dict={x_image: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


() (?, 2) (?, 2)
step 2, training accuracy 0.4
step 102, training accuracy 1
step 202, training accuracy 1


KeyboardInterrupt: 